In [4]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from transformers import AutoTokenizer,AutoModelForCausalLM, BatchEncoding,Trainer, TrainingArguments
import torch
from math import ceil
from transformers import DataCollatorForLanguageModeling
import wandb
from sys import getsizeof
from torch.cuda import device_count
from numba import cuda
from datasets import Dataset

In [2]:
def chunk_array(arr, chunk_size):
    """Chunks an array into chunks of specified size."""
    # For every index in arr stepping by chunk_size, slice arr from that index to index + chunk_size
    return [arr[i:i + chunk_size] for i in range(0, len(arr), chunk_size)]

def tokenize(element,context_length):
    outputs = tokenizer(
        element['text'].to_list(),
        truncation=False
    )
    #print(outputs[2].attention_mask)
    input_batch = []
    for i in range(len(element)):
        temp = outputs[i].ids.copy()
        temp.append(tokenizer.encode(tokenizer.eos_token)[1])
        #outputs[i].ids.append(tokenizer.encode(tokenizer.eos_token)[1])
        input_batch += temp
    print(len(input_batch))
    chunks = chunk_array(input_batch, context_length)
    if len(chunks[-1]) != context_length:
        del chunks[-1]
    #last_chunk = chunks[-1]
    #n_pads = context_length - len(last_chunk)
    #last_chunk += [tokenizer.encode(tokenizer.pad_token)[1]]*n_pads
    
    wrapped_chunks = [BatchEncoding({"input_ids":torch.tensor(chunk), "attention_mask":torch.tensor([1]*context_length)}) for chunk in chunks]
    
    return wrapped_chunks

In [16]:
model_name = 'meta-llama/Llama-2-13b-hf'#'artifacts/checkpoint-cme0pvg4:v2'
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token='')

context_length = 4096
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [51]:
data = pd.read_json("pre_train_data.json",compression=None)
train, test = train_test_split(data,test_size = 0.05,random_state = 42)
chunks = tokenize(test,context_length)
print(chunks[0])

248603
{'input_ids': tensor([    1,  4721,   862,  ...,   262,  6352, 20642]), 'attention_mask': tensor([1, 1, 1,  ..., 1, 1, 1])}


In [53]:
train_chunks = tokenize(train,context_length)
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
out = data_collator([chunks[i] for i in range(8)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

8799651
input_ids shape: torch.Size([8, 4096])
attention_mask shape: torch.Size([8, 4096])
labels shape: torch.Size([8, 4096])


In [23]:
print(len(train_chunks))

2148


In [63]:
print(tokenizer.decode(train_chunks[1123]['input_ids']))

hållet. När drabbat öga 
täcks för försvinner den mest perifera bilden. Undersök: Ögonmotorik, pupillreaktioner (normala), B-glukos, 
BT, Borreliaserologi, SR/CRP, akut neurologremiss om yngre. Diff: Myasteni (kvällstid, ptos), annan 
kranialnervspares, endokrin oftalmopati (exoftalmus), orbitalt tillstånd. Beh: Mikrovaskulär orsak går i spon
regress inom ett par mån. Fresnellprisma på glasögat för eliminera diplopi i primärposition. Täcka för paretis
öga. Pat får ej köra bil! Om pares ej går tillbaka inom 6-12 mån är kir möjligt. Uppföljning tills pares gått i 
regress. Prognos: 100% restitution vid mikroangiopati pga diabetes. 50% vid trauma.  
tan 
kt 
 
2. Troklearispares: Akut. Kongenital, trauma, diabetes, arterioskleros, hypertoni, Borrelia, intrakraniell process, 
temporalisarterit. Tvångshuvudhållning (alltid håller huvudet lite på sned åt friska sidan för minimera diplopin). 
Bielschowskys head tilt-test: Luta pats huvud åt andra hållet  paretiskt öga åker uppåt  diplopi. U

In [7]:
prompt1 = '''[INST] <<SYS>>
Du är en hjälpsam medicinsk assistent som hjälper läkare och sjuksköterskor genom att sammanfatta information om patienter.
Svara på svenska.
<</SYS>>
Nedan ges anamnes för en patient under en dag
<anamnes>'''

prompt2 = '''
</anamnes>
Du ska plocka ut information som passar i mallen nedan. Undvik onödig information och plocka endast ut sådant som rör varje rubrik. Om relevant information saknas så lämnar du rubriken tom.
Formattera ditt svar enligt mallen. Ingen information kan finnas under flera rubriker.
<mall>
*Sjukdomshistoria (Patientens diagnoser, sjukdomshistorik och riskfaktorer (t.ex. sjukdomar i familjen))*

*Sökorsaker (Patientens symtom och/eller datum för ingrepp)*

*Åtgärder (Planerade undersökningar, behandlingar och åtgärder)*
</mall>
[/INST]'''
data = pd.read_parquet("synthetic_229_corrected.parquet")

formatted = []
for index, row in data.iterrows():
    #print(row)
    elem = {'text':f"<s> {prompt1}\n{row['description']}{prompt2}\n{row['summary']} </s>"}
    formatted.append(elem)
    

train, test = train_test_split(formatted,test_size = 0.05,random_state = 42)
train = Dataset.from_list(train)
test = Dataset.from_list(test)

In [25]:
lengths = 0
for data in test:
    #print(len(tokenizer.encode(data['text'])[1:]))
    lengths += len(tokenizer.encode(data['text'])[1:])
print(lengths)

12502
